# DeepTurkish

This notebook is for training, evaluating and testing the trained DeepSpeech2 implementation.

In [ ]:
import os

import utilities.utilities as utils
from model.data_loader import pad_collate_fn
from decoders import decoders
from train import main

## Dataset Directories

Choose a dataset, and the spectrogram type for training the network.

In [ ]:
dataset_name = "METUbet"
spectrogram_type = "dB"

spectrogram_dir, df_dir0, df_dir1, alphabet = utils.dataset_pointers(dataset_name, spectrogram_type)

## Parameters

In [ ]:
project_name = "try"

# leave an empty string for new model
#model_dir = os.path.join("data","Models and Losses","try","00_28__30_01_2021","00_28__30_01_2021.pt")
model_dir = ""

hyperparameters = {
    "n_cnn_layers": 1,
    "n_rnn_layers": 1,
    "rnn_dim": 4,
    "n_class": len(alphabet),
    "N_fft": 512,
    "stride":2,
    "dropout": 0.1,
    "learning_rate": 5e-4,   
    "epochs": 5,
    'batch_size':16,
    'SortaGrad': True,
    'model_dir': model_dir
}


loader_parameters = {'batch_size': hyperparameters['batch_size'],
                    'shuffle': True,
                    'collate_fn': pad_collate_fn,          
                    'pin_memory': True, # Faster cuda transfer if multiprocess, create tensors directly in cuda if single process
                    'num_workers': 0 # increase if you hav enough gpu memory
                    }


data_parameters = {'dataframe_dir_train': df_dir0, # directory of the dataframes,
                   'dataframe_dir_test': df_dir1,
                   'train_dir': spectrogram_dir,
                   'test_dir': spectrogram_dir,
                   'alphabet': alphabet, 
                   'blank': 0, #idx of blank symbol
                   'batch_size': hyperparameters['batch_size'],
                   'split_ratio': 0.8,
                   'loader_parameters': loader_parameters
                   }

## Decoder

Choose a decoder for decoding the ctc output matrix.

In [ ]:
# Argmax decoder
decoder = decoders.Argmax_decoder(alphabet, data_parameters['blank'])

In [ ]:
# BeamSearch Decoder
LM_text_name="NN_datasets_sentences"
beam_width = 3
prune_threshold = -7 # log(0.001)

decoder = decoders.BeamSearch_decoder(alphabet, data_parameters['blank'], beam_width, prune_threshold, LM_text_name)

In [ ]:
# LexiconSearch
tolerance = 1

# choose an apprroximator for the Lexicon Search algorithm
BW = 2
prune = -7 # = log(0.001)
LM_text_name="NN_datasets_sentences"

approximator_properties = ('BeamSearch+LM',data_parameters['blank'], BW, prune, LM_text_name)

decoder = decoders.LexiconSearch_decoder(alphabet, tolerance, LM_text_name, approximator_properties)

## Train, Validate and Test

The cell below will use the parameters to train the network and the chosen decoder to decode the outputs. After each training epoch it will do a validation run on the validation set and when the training is complete it will test the model on the test set.

In [ ]:
main(hyperparameters, data_parameters, decoder, project_name)